<a href="https://colab.research.google.com/github/vachkim/LECTURE/blob/master/MINING/Lec_EMOTION_ANALYSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 0.머리말
- 감정분석을 활용해 새로운 매일에서 happy weekend가 포함되어 있을 경우의
- 긍정(1)/부정(0)메시지 여부를 알아보자

### 1.데이터 불러오기

In [ ]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd drive/'My Drive'/'Colab Notebooks'

/content/drive/My Drive/Colab Notebooks


In [ ]:
%ls

'모두의 데이터 분석'/       강의_감정분석.ipynb       SamsungCard.ipynb
 0826_Class.ipynb           강의_한글마이닝.ipynb     samsung_data/
 강의_BeautifulSoup.ipynb   데이터시각화기초.ipynb    Seaborn_시각화.ipynb
 deeplearning_data/         인구공공데이터.ipynb      강의_Selenium.ipynb
 강의_판다스.ipynb          기온공공데이터.ipynb      titanic_data/
 강의_파이썬.ipynb          machinelearning_data/     Titanic_기초분석.ipynb
 강의_넘파이.ipynb          Matplotlib_시각화.ipynb   weather_data/
 강의_코딩실습.ipynb        naivebayes_example.csv
 강의_영어마이닝.ipynb      SamsungCard_분석.ipynb


In [ ]:
import pandas as pd
data = pd.read_csv('naivebayes_example.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   mail    6 non-null      object
 1   label   6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 224.0+ bytes


In [ ]:
data

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


### 2.인덱스 부여

In [ ]:
mail = list(data['mail'])
split = []
for sentence in mail:
  split.append(sentence.split(' '))
split

[['i', 'love', 'you'],
 ['love', 'happy', 'weekend'],
 ['bore', 'work', 'job'],
 ['i', 'hate', 'you'],
 ['bore', 'weekend'],
 ['happy', 'together']]

In [ ]:
from collections import defaultdict
idx = defaultdict(lambda: len(idx))
for sentence in split:
  for token in sentence:
    idx[token]
idx

defaultdict(<function __main__.<lambda>>,
            {'bore': 5,
             'happy': 3,
             'hate': 8,
             'i': 0,
             'job': 7,
             'love': 1,
             'together': 9,
             'weekend': 4,
             'work': 6,
             'you': 2})

In [ ]:
pos = data[data['label']==1]
neg = data[data['label']==0]
pos_mail = list(pos['mail'])
neg_mail = list(neg['mail'])

In [ ]:
pos_split = []
neg_split = []
for sentence in pos_mail:
  pos_split.append(sentence.split(' '))
for sentence in neg_mail:
  neg_split.append(sentence.split(' '))

### 3.빈도 수 파악

In [ ]:
import numpy as np
pos_word = []
neg_word = []

for i, doc in enumerate(pos_split):
  word = np.zeros(len(idx),dtype=int)
  for token in doc:
    word[idx[token]] += 1 #토큰의 위치가 같으면
  pos_word.append(word.tolist())

for i, doc in enumerate(neg_split):
  word = np.zeros(len(idx),dtype=int)
  for token in doc:
    word[idx[token]] += 1 #토큰의 위치가 같으면
  neg_word.append(word.tolist())

print(pos_word); print(neg_word)

[[1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]]
[[0, 0, 0, 0, 0, 1, 1, 1, 0, 0], [1, 0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]]


In [ ]:
pos_total=np.sum(pos_word,0)
neg_total=np.sum(neg_word,0)

In [ ]:
sort = sorted((value,key) for key, value in idx.items())
vocab = []
for item in sort:
  vocab.append(item[1])

In [ ]:
fre = pd.DataFrame([pos_total,neg_total],index=['pos','neg'],columns=vocab).T
fre

,pos,neg
i,1,1
love,2,0
you,1,1
happy,2,0
weekend,1,1
bore,0,2
work,0,1
job,0,1
hate,0,1
together,1,0


### 4.조건부확률 구하기

In [ ]:
def pos_per(x):
  per1 = (fre.loc[x,'pos']+1)/(sum(fre['pos'])+len(vocab))
  return per1

In [ ]:
def neg_per(x):
  per2 = (fre.loc[x,'neg']+1)/(sum(fre['neg'])+len(vocab))
  return per2

### 5.감정분석 수행하기

In [ ]:
def guess_pos(x,y):
  import math
  numerator = math.exp(math.log(pos_per(x))+math.log(pos_per(y))+math.log(0.5))
  denominator1 = math.exp(math.log(pos_per(x))+math.log(pos_per(y))+math.log(0.5))
  denominator2 = math.exp(math.log(neg_per(x))+math.log(neg_per(y))+math.log(0.5))
  act_pos = numerator/(denominator1+denominator2)
  return act_pos

In [ ]:
def guess_neg(x,y):
  import math
  numerator = math.exp(math.log(neg_per(x))+math.log(neg_per(y))+math.log(0.5))
  denominator1 = math.exp(math.log(neg_per(x))+math.log(neg_per(y))+math.log(0.5))
  denominator2 = math.exp(math.log(pos_per(x))+math.log(pos_per(y))+math.log(0.5))
  act_pos = numerator/(denominator1+denominator2)
  return act_pos

In [ ]:
guess_pos('happy','weekend')

0.7500000000000001

In [ ]:
guess_neg('happy','weekend')

0.24999999999999997

### 6.Sklearn을 활용한 감정분석 수행

In [ ]:
X_train = list(data['mail'])
Y_train = list(data['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
clf = MultinomialNB().fit(X_train_counts, Y_train)

In [ ]:
print(clf.predict(count_vect.transform(['happy weekend'])))
print(clf.predict_proba(count_vect.transform(['happy weekend'])))

[1]
[[0.25 0.75]]
